In [1]:
import os
import nltk
import re
import pyprind
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import sys
import pandas as pd
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
    TfidfVectorizer,
)
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
import numpy as np

nltk.download("stopwords")

basepath = "/home/itz-amethyst/dev/ML_Space/ML_With_Pytorch_Scikit_learn_Sebastian/ch08/aclImdb"
labels = {"pos": 1, "neg": 0}
pybar = pyprind.ProgBar(50000, stream=sys.stdout)
rows = []
for s in ("test", "train"):
    for l in ("pos", "neg"):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file)) as infile:
                txt = infile.read()
            rows.append((txt, labels[l]))
            pybar.update()

df = pd.DataFrame(rows, columns=['review', 'sentiment'])
print(os.listdir(basepath))

[nltk_data] Downloading package stopwords to /home/itz-
[nltk_data]     amethyst/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0% [#                             ] 100% | ETA: 00:00:04

0% [##                            ] 100% | ETA: 00:00:04

0% [###                           ] 100% | ETA: 00:00:04

0% [####                          ] 100% | ETA: 00:00:04

...

0% [############################# ] 100% | ETA: 00:00:00

0% [##############################] 100% | ETA: 00:00:00

0% [##############################] 100% | ETA: 00:00:00


Total time elapsed: 00:00:04


['test', 'train', 'imdbEr.txt', 'imdb.vocab', 'README']


In [2]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv("/home/itz-amethyst/dev/ML_Space/ML_With_Pytorch_Scikit_learn_Sebastian/ch08/movie_data.csv", index=False)
df.head()
df = pd.read_csv("/home/itz-amethyst/dev/ML_Space/ML_With_Pytorch_Scikit_learn_Sebastian/ch08/movie_data.csv")
df.shape
# Not needed
# df = df.rename(columns={"0": "review", "1": "sentiment"})
# df.head(3)

(50000, 2)

In [3]:
count = CountVectorizer()
docs = np.array(
    [
        "The sun is shining",
        "The weather is sweet",
        "The sun is shining, the weather is sweet, and one and one is two",
    ]
)
bag = count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())
tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]
[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [4]:
def preprocess(text):
    text = re.sub("<[^>]*", "", text)
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
    text = re.sub("[\W]+", " ", text.lower()) + " ".join(emoticons).replace("-", "")
    return text


df.loc[9, "review"][-50:]
preprocess(df.loc[9, "review"][-50:])
preprocess("</a>this :) is :( a test :-)!")
df["review"] = df["review"].apply(preprocess)

<>:3: SyntaxWarning: invalid escape sequence '\)'
<>:4: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\)'
<>:4: SyntaxWarning: invalid escape sequence '\W'
/tmp/ipykernel_1398894/2783076914.py:3: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
/tmp/ipykernel_1398894/2783076914.py:4: SyntaxWarning: invalid escape sequence '\W'
  text = re.sub("[\W]+", " ", text.lower()) + " ".join(emoticons).replace("-", "")


In [5]:
porter = PorterStemmer()
stop = stopwords.words("english")


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]


def tokenizer(text):
    return text.split()


txt = "runners like running and thus they run and so on"
[w for w in tokenizer_porter(txt) if w not in stop]
# tokenizer_porter(txt)

['runner', 'like', 'run', 'thu', 'run']

In [6]:
# X_train = df.iloc[:25000]["review"].values
# X_test = df.iloc[25000:]["review"].values
# y_test = df.iloc[25000:]["sentiment"].values
# y_train = df.iloc[:25000]["sentiment"].values
X_train = df.iloc[:25000]["review"].to_numpy(dtype=str)
X_test = df.iloc[25000:]["review"].to_numpy(dtype=str)
y_test = df.iloc[25000:]["sentiment"].to_numpy(dtype=int)

y_train = df.iloc[:25000]["sentiment"].to_numpy(dtype=int)
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

small_param_grid = [
    {
        "vect__ngram_range": [(1, 1)],
        "vect__stop_words": [None],
        "vect__tokenizer": [tokenizer, tokenizer_porter],
        "clf__penalty": ["l2"],
        "clf__C": [1.0, 10.0],
    },
    {
        "vect__ngram_range": [(1, 1)],
        "vect__stop_words": [stop, None],
        "vect__tokenizer": [tokenizer],
        "vect__use_idf": [False],
        "vect__norm": [None],
        "clf__penalty": ["l2"],
        "clf__C": [1.0, 10.0],
    },
]
lr_tdidf = Pipeline([("vect", tfidf), ("clf", LogisticRegression(solver="liblinear"))])
gs_lr_tfidf = GridSearchCV(
    lr_tdidf, small_param_grid, scoring="accuracy", cv=5, verbose=2, n_jobs=-1
)
gs_lr_tfidf.fit(X_train, y_train)
print(f"Best parameters set: {gs_lr_tfidf.best_params_}")
print(f"CV accuracy: {gs_lr_tfidf.best_score_:.3f}")
clf = gs_lr_tfidf.best_estimator_
print(f"Test accuracy: {clf.score(X_test, y_test):.3f}")

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


...

/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


/home/itz-amethyst/venvs/global-venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


Best parameters set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7341d7919bc0>}
CV accuracy: 0.897


Test accuracy: 0.899


In [7]:
stop = stopwords.words("english")
def tokenizer(text):
    text = re.sub("<[^>]*", "", text)
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
    text = re.sub("[\W]+", " ", text.lower()) + " ".join(emoticons).replace("-", "")
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized 

def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # to skip the header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

next(stream_docs(path="/home/itz-amethyst/dev/ML_Space/ML_With_Pytorch_Scikit_learn_Sebastian/ch08/movie_data.csv"))

<>:4: SyntaxWarning: invalid escape sequence '\)'
<>:5: SyntaxWarning: invalid escape sequence '\W'
<>:4: SyntaxWarning: invalid escape sequence '\)'
<>:5: SyntaxWarning: invalid escape sequence '\W'
/tmp/ipykernel_1398894/546467502.py:4: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
/tmp/ipykernel_1398894/546467502.py:5: SyntaxWarning: invalid escape sequence '\W'
  text = re.sub("[\W]+", " ", text.lower()) + " ".join(emoticons).replace("-", "")


('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [8]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)
clf = SGDClassifier(loss='log_loss', random_state=1) # we initialized logistic regression classifier by setting loss to log
doc_stream = stream_docs(path='/home/itz-amethyst/dev/ML_Space/ML_With_Pytorch_Scikit_learn_Sebastian/ch08/movie_data.csv')

In [9]:
pbar = pyprind.ProgBar(45, stream=sys.stdout)
classes = np.array([0,1])
# do not run this twice unless we get ran out of documents to process for test stage
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f"Accuracy: {clf.score(X_test, y_test):.3f}")
clf = clf.partial_fit(X_test, y_test)

0% [#                             ] 100% | ETA: 00:00:20

0% [##                            ] 100% | ETA: 00:00:19

0% [###                           ] 100% | ETA: 00:00:18

0% [####                          ] 100% | ETA: 00:00:17

0% [#####                         ] 100% | ETA: 00:00:17

...

0% [############################# ] 100% | ETA: 00:00:00

0% [##############################] 100% | ETA: 00:00:00

0% [##############################] 100% | ETA: 00:00:00


Total time elapsed: 00:00:21


Accuracy: 0.868


In [10]:
df = pd.read_csv('/home/itz-amethyst/dev/ML_Space/ML_With_Pytorch_Scikit_learn_Sebastian/ch08/movie_data.csv')
df.head()
count = CountVectorizer(stop_words='english', max_df=.1,max_features=5000)
X = count.fit_transform(df['review'].values)

In [11]:
lda = LatentDirichletAllocation(n_components=10, random_state=123, learning_method='batch')
X_topics = lda.fit_transform(X)
lda.components_.shape

n_top_words = 5
feature_names = count.get_feature_names_out()
# will print the most important words from each topic 
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1)}:')
    print(' '.join([feature_names[i] for i in topic.argsort() [:-n_top_words - 1:-1]]))

# horror category is topic number 6 with index 0 is number 5
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f"Horror Movie #{(iter_idx + 1)}: \n")
    print(df['review'][movie_idx][:300], '...')

Topic 1:
horror worst script effects budget
Topic 2:
dvd watched video music guy
Topic 3:
war american series history documentary
Topic 4:
game killer murder thriller crime
Topic 5:
kids comedy episode series school
Topic 6:
family woman mother beautiful feel
Topic 7:
role performance comedy john plays
Topic 8:
action horror john effects dr
Topic 9:
book version original read music
Topic 10:
action wife father police james
Horror Movie #1: 

"Come Undone" appears to elicit a lot of opinions among the contributors to this forum. Granted, it's a film that promises a take on gay life, as most viewers expect and somehow, it gets away from that promise into an introspective view at a young man's soul. The film has a way of staying with us ev ...
Horror Movie #2: 

The subject of children being terminally ill is difficult and saddening but 'The Cure' successfully portrays the idea that it doesn't have to be all doom and gloom and, if anything, children need to have hope and delight in their 